In [1]:
import json
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
with open('Data/assumptions.json') as f:
    data = json.load(f)

Convert data into a tabular format where each level in the JSON becomes a column in the DataFrame

Creates a clean DataFrame with Category (A/B), Risk (HR/LR), Type (Investment/cash), and Value columns

In [3]:
# Initialize an empty list to store our flattened data
flattened_data = []

In [4]:
# Iterate through the nested structure
for category, category_data in data["Assumptions"].items():
    for risk, risk_data in category_data.items():
        for asset_type, value in risk_data.items():
            # Append to our list
            flattened_data.append({
                "Category": category,
                "Risk": risk,
                "Type": asset_type,
                "Value": value
            })
df = pd.DataFrame(flattened_data)

In [5]:
df.head()

,Category,Risk,Type,Value
0,A,HR,Investment,1.0
1,A,HR,cash,2.0
2,A,LR,Investment,3.0
3,A,LR,cash,4.0
4,B,HR,Investment,5.0


To save your DataFrame to a database, you can use pandas' built-in to_sql() method. Here's how to do it with SQLite (simplest option)

In [6]:
# Create SQLite database engine
engine = create_engine('sqlite:///assumptions.db')

In [7]:
# Save DataFrame to SQLite
df.to_sql('assumptions', engine, if_exists='replace', index=False)

8

In [8]:
print(pd.read_sql('SELECT * FROM assumptions', engine))

  Category Risk        Type  Value
0        A   HR  Investment    1.0
1        A   HR        cash    2.0
2        A   LR  Investment    3.0
3        A   LR        cash    4.0
4        B   HR  Investment    5.0
5        B   HR        cash    6.0
6        B   LR  Investment    7.0
7        B   LR        cash    8.0


Alter value

In [9]:
with engine.connect() as conn:
    # Execute an UPDATE statement
    conn.execute(text("""
        UPDATE assumptions
        SET Value = 10.0
        WHERE Risk = 'HR' AND Type = 'Investment'
    """))
    conn.commit()

In [10]:
print(pd.read_sql('SELECT * FROM assumptions', engine))

  Category Risk        Type  Value
0        A   HR  Investment   10.0
1        A   HR        cash    2.0
2        A   LR  Investment    3.0
3        A   LR        cash    4.0
4        B   HR  Investment   10.0
5        B   HR        cash    6.0
6        B   LR  Investment    7.0
7        B   LR        cash    8.0


In [11]:
df2 = pd.read_sql('SELECT * FROM assumptions', engine)

In [12]:
df2.head()

,Category,Risk,Type,Value
0,A,HR,Investment,10.0
1,A,HR,cash,2.0
2,A,LR,Investment,3.0
3,A,LR,cash,4.0
4,B,HR,Investment,10.0


To convert the altered database table back to the original JSON format, read the data and reconstruct the nested structure

In [13]:
new_json = {"Assumptions": {}}

In [14]:
# Iterate through the DataFrame to build the nested structure
for _, row in df2.iterrows():
    category = row['Category']
    risk = row['Risk']
    asset_type = row['Type']
    value = row['Value']

    # Create nested structure
    if category not in new_json["Assumptions"]:
        new_json["Assumptions"][category] = {}

    if risk not in new_json["Assumptions"][category]:
        new_json["Assumptions"][category][risk] = {}

    # Set the value
    new_json["Assumptions"][category][risk][asset_type] = value

Save to a new JSON file

In [15]:
with open('Data/assumptions_updated.json', 'w') as f:
    json.dump(new_json, f, indent=2)